In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [19]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/temperature", inferSchema=True, header=True)
df.createOrReplaceTempView("temperature")

In [20]:
df_batch_result = spark.sql("SELECT * FROM temperature")

df_batch_result.toPandas()

,stations_id,Datum,Avg_Lufttemperatur_2m_Hoehe,Max_Lufttemperatur_2m_Hoehe,Min_Lufttemperatur_2m_Hoehe,Min_Lufttemperatur_5cm_Hoehe
0,3631,1994-01-01,3.0,4.7,2.5,1.7
1,3631,1994-01-02,0.8,2.9,0.0,0.2
2,3631,1994-01-03,4.6,5.3,0.7,0.5
3,3631,1994-01-04,5.6,6.5,4.1,3.4
4,3631,1994-01-05,4.9,6.3,3.5,2.6
...,...,...,...,...,...,...
151114,850,2023-12-27,4.2,NaN,NaN,NaN
151115,850,2023-12-28,10.1,NaN,NaN,NaN
151116,850,2023-12-29,9.3,NaN,NaN,NaN
151117,850,2023-12-30,7.1,NaN,NaN,NaN


# AVG, MIN, MAX

In [23]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        AVG(Min_Lufttemperatur_5cm_Hoehe) as avg_Lufttemperatur_Erdboden
    FROM 
        temperature
    GROUP BY 
        Jahr, Monat
    """
)

# In Pandas DataFrame umwandeln
df_batch_result.toPandas()

,Jahr,Monat,avg_Lufttemperatur_Erdboden
0,1997,11,0.909444
1,2022,10,6.377957
2,1995,12,-4.131965
3,2005,5,6.328536
4,1998,7,11.862757
...,...,...,...
355,2015,5,4.664268
356,1996,8,11.759677
357,2007,3,0.143280
358,2021,4,-0.976389


In [24]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/Analitic_Results/Temperature/avg", mode="overwrite")